In [1]:
import numpy as np
import sys, os

# file_path = os.path.dirname(os.path.abspath(__file__)) # this is for the .py script but does not work in a notebook
file_path = os.path.dirname(os.path.abspath(''))
sys.path.append(file_path)
# sys.path.append(os.path.join(file_path, "grid-graph/python/bin"))
# sys.path.append(os.path.join(file_path, "parallel-cut-pursuit/python/wrappers"))

# Data loading

In [11]:
from time import time
import glob
from torch_geometric.data import Data
from superpoint_transformer.datasets.kitti360 import read_kitti360_window
from superpoint_transformer.datasets.kitti360_config import KITTI360_NUM_CLASSES

# DATA_ROOT
DATA_ROOT = '/media/drobert-admin/DATA2'
# DATA_ROOT = '/var/data/drobert

i_window = 0
all_filepaths = sorted(glob.glob(DATA_ROOT + '/datasets/kitti360/shared/data_3d_semantics/*/static/*.ply'))
filepath = all_filepaths[i_window]

start = time()
data = read_kitti360_window(filepath, semantic=True, instance=False, remap=True)
print(f'Loading data {i_window+1}/{len(all_filepaths)}: {time() - start:0.3f}s')
print(f'Number of loaded points: {data.num_nodes} ({data.num_nodes // 10**6:0.2f}M)')

Loading data 1/342: 0.038s
Number of loaded points: 3201318 (3.00M)


# Voxelization

In [12]:
# voxel = 0.05
voxel = 0.5

### Pytorch Geometric

In [4]:
from torch_geometric.nn.pool import voxel_grid

start = time()
data_sub = voxel_grid(data.pos, size=0.1)
print(f'Data  voxelization at {voxel}m: {time() - start:0.3f}s')
print(f'Number of sampled points: {data_sub.shape[0]} ({data_sub.shape[0] // 10**6:0.2f}M, {100 * data_sub.shape[0] / data.num_nodes:0.1f}%)')

Data  voxelization at 0.05m: 0.133s
Number of sampled points: 3201318 (3.00M, 100.0%)


### TorchPoints3D

In [4]:
import torch
import re
from torch_geometric.nn.pool import voxel_grid
from torch_cluster import grid_cluster
from torch_scatter import scatter_mean, scatter_add
from torch_geometric.nn.pool.consecutive import consecutive_cluster
MAPPING_KEY = 'mapping_index'

# Label will be the majority label in each voxel
_INTEGER_LABEL_KEYS = ["y", "instance_labels"]

def group_data(data, cluster=None, unique_pos_indices=None, mode="last", skip_keys=[]):
    """ Group data based on indices in cluster.
    The option ``mode`` controls how data gets agregated within each cluster.

    Parameters
    ----------
    data : Data
        [description]
    cluster : torch.Tensor
        Tensor of the same size as the number of points in data. Each element is the cluster index of that point.
    unique_pos_indices : torch.tensor
        Tensor containing one index per cluster, this index will be used to select features and labels
    mode : str
        Option to select how the features and labels for each voxel is computed. Can be ``last`` or ``mean``.
        ``last`` selects the last point falling in a voxel as the representent, ``mean`` takes the average.
    skip_keys: list
        Keys of attributes to skip in the grouping
    """

    assert mode in ["mean", "last"]
    if mode == "mean" and cluster is None:
        raise ValueError("In mean mode the cluster argument needs to be specified")
    if mode == "last" and unique_pos_indices is None:
        raise ValueError("In last mode the unique_pos_indices argument needs to be specified")

    num_nodes = data.num_nodes
    for key, item in data:
        if bool(re.search("edge", key)):
            raise ValueError("Edges not supported. Wrong data type.")
        if key in skip_keys:
            continue

        if torch.is_tensor(item) and item.size(0) == num_nodes:
            if mode == "last" or key == "batch" \
                    or key == SaveOriginalPosId.KEY \
                    or key == MAPPING_KEY:
                data[key] = item[unique_pos_indices]
            elif mode == "mean":
                is_item_bool = item.dtype == torch.bool
                if is_item_bool:
                    item = item.int()
                if key in _INTEGER_LABEL_KEYS:
                    item_min = item.min()
                    item = torch.nn.functional.one_hot(item - item_min)
                    item = scatter_add(item, cluster, dim=0)
                    data[key] = item.argmax(dim=-1) + item_min
                else:
                    data[key] = scatter_mean(item, cluster, dim=0)
                if is_item_bool:
                    data[key] = data[key].bool()
    return data

class SaveOriginalPosId:
    """ Transform that adds the index of the point to the data object
    This allows us to track this point from the output back to the input data object
    """

    KEY = "origin_id"

    def __init__(self, key=None):
        self.KEY = key if key is not None else self.KEY

    def _process(self, data):
        if hasattr(data, self.KEY):
            return data

        setattr(data, self.KEY, torch.arange(0, data.pos.shape[0]))
        return data

    def __call__(self, data):
        if isinstance(data, list):
            data = [self._process(d) for d in data]
        else:
            data = self._process(data)
        return data

    def __repr__(self):
        return self.__class__.__name__
    
class GridSampling3D:
    """ Clusters points into voxels with size :attr:`size`.
    Parameters
    ----------
    size: float
        Size of a voxel (in each dimension).
    quantize_coords: bool
        If True, it will convert the points into their associated sparse
        coordinates within the grid and store the value into a new
        `coords` attribute.
    mode: string:
        The mode can be either `last` or `mean`.
        If mode is `mean`, all the points and their features within a
        cell will be averaged. If mode is `last`, one random points per
        cell will be selected with its associated features.
    setattr_full_pos: bool
        If True, the input point positions will be saved into a new
        'full_pos' attribute. This memory-costly step may reveal
        necessary for subsequent local feature computation.
    """

    def __init__(self, size, quantize_coords=False, mode="mean", verbose=False,
                 setattr_full_pos=False):
        self._grid_size = size
        self._quantize_coords = quantize_coords
        self._mode = mode
        self._setattr_full_pos = setattr_full_pos
        if verbose:
            log.warning(
                "If you need to keep track of the position of your points, use "
                "SaveOriginalPosId transform before using GridSampling3D.")

            if self._mode == "last":
                log.warning(
                    "The tensors within data will be shuffled each time this "
                    "transform is applied. Be careful that if an attribute "
                    "doesn't have the size of num_points, it won't be shuffled")

    def _process(self, data):
        if self._mode == "last":
            data = shuffle_data(data)

        full_pos = data.pos
        coords = torch.round((data.pos) / self._grid_size)
        if "batch" not in data:
            cluster = grid_cluster(coords, torch.ones(3, device=coords.device))
        else:
            cluster = voxel_grid(coords, data.batch, 1)
        cluster, unique_pos_indices = consecutive_cluster(cluster)

        data = group_data(data, cluster, unique_pos_indices, mode=self._mode)
        if self._quantize_coords:
            data.coords = coords[unique_pos_indices].int()

        data.grid_size = torch.tensor([self._grid_size])

        # Keep track of the initial full-resolution point cloud for
        # later use. Typically needed for local features computation.
        # However, for obvious memory-wary considerations, it is
        # recommended to delete the 'full_pos' attribute as soon as it
        # is no longer needed.
        if self._setattr_full_pos:
            data.full_pos = full_pos

        return data

    def __call__(self, data):
        if isinstance(data, list):
            data = [self._process(d) for d in data]
        else:
            data = self._process(data)
        return data

    def __repr__(self):
        return "{}(grid_size={}, quantize_coords={}, mode={})".format(
            self.__class__.__name__, self._grid_size, self._quantize_coords, self._mode
        )

In [6]:
# CPU
start = time()
data_sub = GridSampling3D(size=voxel)(data.clone())
print(f'Data  voxelization at {voxel}m: {time() - start:0.3f}s')
print(f'Number of sampled points: {data_sub.num_nodes} ({data_sub.num_nodes // 10**6:0.2f}M, {100 * data_sub.num_nodes / data.num_nodes:0.1f}%)')

Data  voxelization at 0.05m: 2.559s
Number of sampled points: 2480151 (2.00M, 77.5%)


In [26]:
# GPU
torch.cuda.synchronize()
start = time()
data_sub = GridSampling3D(size=voxel)(data.clone().cuda()).cpu()
torch.cuda.synchronize()
print(f'Data  voxelization at {voxel}m: {time() - start:0.3f}s')
print(f'Number of sampled points: {data_sub.num_nodes} ({data_sub.num_nodes // 10**6:0.2f}M, {100 * data_sub.num_nodes / data.num_nodes:0.1f}%)')

Data  voxelization at 0.05m: 0.130s
Number of sampled points: 2480168 (2.00M, 77.5%)


### SPG C implem

In [15]:
import superpoint_transformer.partition.utils.libpoint_utils as point_utils

# WARNING: the pruning must know the number of classes. All labels are 
# offset to account for the -1 unlabeled points !
start = time()
xyz, rgb, labels, dump = point_utils.prune(data.pos.float().numpy(), voxel, (data.rgb * 255).byte().numpy(), data.y.byte().numpy() + 1, np.zeros(1, dtype='uint8'), KITTI360_NUM_CLASSES + 1, 0)
print(f'Data  voxelization at {voxel}m: {time() - start:0.3f}s')
print(f'Number of sampled points: {xyz.shape[0]} ({xyz.shape[0] // 10**6:0.2f}M, {100 * xyz.shape[0] / data.num_nodes:0.1f}%)')

Data  voxelization at 0.05m: 7.307s
Number of sampled points: 2479935 (2.00M, 77.5%)


So it seems the C-based voxelization is not that fast. Can we somehow make it faster with more CPU cores ? Otherwise, will fallback to a custom implementation based on TP3D or PyG and keeping track of the in-voxel label distribution.

And even increasing the number of CPU cores (on AI4GEO) gave the same results.

The fastest is GPU-based TP3D-based computation.

### Final

In [10]:
# GPU
torch.cuda.synchronize()
start = time()
n_in = data.num_nodes
data = GridSampling3D(size=voxel)(data.clone().cuda()).cpu()
torch.cuda.synchronize()
print(f'Data  voxelization at {voxel}m: {time() - start:0.3f}s')
print(f'Number of sampled points: {data.num_nodes} ({data.num_nodes // 10**6:0.2f}M, {100 * data.num_nodes / n_in:0.1f}%)')

Data  voxelization at 0.5m: 0.047s
Number of sampled points: 96291 (0.00M, 3.0%)


# Neighbour search

In [5]:
x = data.pos
k = 10

### Sklearn

In [6]:
from sklearn.neighbors import KDTree

start = time()
kdt = KDTree(x.numpy(), leaf_size=30, metric='euclidean')
neighbors = kdt.query(x.numpy(), k=k, return_distance=False)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 15.029s


### FAISS-GPU
```
conda install -c pytorch faiss-gpu cudatoolkit=10.2
pip install faiss-gpu cudatoolkit==10.2
```

In [ ]:
import faiss

def find_neighbours(x, y, k=10, ncells=None, nprobes=10):
    # if batch_x is not None or batch_y is not None:
    #     raise NotImplementedError(
    #         "FAISSGPUKNNNeighbourFinder does not support batches yet")

    x = x.view(-1, 1) if x.dim() == 1 else x
    y = y.view(-1, 1) if y.dim() == 1 else y
    x, y = x.contiguous(), y.contiguous()

    # FAISS-GPU consumes numpy arrays
    x_np = x.cpu().numpy()
    y_np = y.cpu().numpy()

    # Initialization
    n_fit = x_np.shape[0]
    d = x_np.shape[1]
    gpu = faiss.StandardGpuResources()

    # Heuristics to prevent k from being too large
    k_max = 1024
    k = min(k, n_fit, k_max)

    # Heuristic to parameterize the number of cells for FAISS index,
    # if not provided
    if ncells is None:
        f1 = 3.5 * np.sqrt(n_fit)
        f2 = 1.6 * np.sqrt(n_fit)
        if n_fit > 2 * 10 ** 6:
            p = 1 / (1 + np.exp(2 * 10 ** 6 - n_fit))
        else:
            p = 0
        ncells = int(p * f1 + (1 - p) * f2)

    # Building a GPU IVFFlat index + Flat quantizer
    torch.cuda.empty_cache()
    quantizer = faiss.IndexFlatL2(d)  # the quantizer index
    index = faiss.IndexIVFFlat(quantizer, d, ncells, faiss.METRIC_L2)  # the main index
    gpu_index_flat = faiss.index_cpu_to_gpu(gpu, 0, index)  # pass index it to GPU
    gpu_index_flat.train(x_np)  # fit the cells to the training set distribution
    gpu_index_flat.add(x_np)

    # Querying the K-NN
    gpu_index_flat.setNumProbes(nprobes)
    return torch.LongTensor(gpu_index_flat.search(y_np, k)[1]).to(x.device)

start = time()
out = find_neighbours(x, x, k=k, ncells=None, nprobes=10)
print(f'Neighbor search: {time() - start:0.3f}s')

### PyKeOps
```
pip install pykeops
```

In [11]:
from pykeops.torch import LazyTensor

start = time()
# K-NN search with KeOps. If the number of points is greater
# than 16 millions, KeOps requires double precision.
xyz_query = x.contiguous()
xyz_search = x.contiguous()
if xyz_search.shape[0] > 1.6e7:
    xyz_query_keops = LazyTensor(xyz_query[:, None, :].double())
    xyz_search_keops = LazyTensor(xyz_search[None, :, :].double())
else:
    xyz_query_keops = LazyTensor(xyz_query[:, None, :])
    xyz_search_keops = LazyTensor(xyz_search[None, :, :])
d_keops = ((xyz_query_keops - xyz_search_keops) ** 2).sum(dim=2)
neighbors = d_keops.argKmin(k, dim=1)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 39.054s


### FLANN
```
conda install -c conda-forge pyflann -y
```

In [8]:
import pyflann

start = time()
flann = pyflann.FLANN()
result, dists = flann.nn(x.numpy(), x.numpy(), k, algorithm="kmeans", branching=32, iterations=7, checks=16)
print(f'Neighbor search: {time() - start:0.3f}s')

ImportError: Cannot load dynamic library. Did you compile FLANN?

### Pytorch Geometric

In [13]:
from torch_geometric.nn import knn

start = time()
out = knn(x, x, k, batch_x=None, batch_y=None, num_workers=1)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 9.466s


In [17]:
start = time()
out = knn(x, x, k, batch_x=None, batch_y=None, num_workers=2)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 9.215s


In [14]:
start = time()
out = knn(x, x, k, batch_x=None, batch_y=None, num_workers=4)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 9.076s


In [16]:
start = time()
out = knn(x, x, k, batch_x=None, batch_y=None, num_workers=8)
print(f'Neighbor search: {time() - start:0.3f}s')

Neighbor search: 9.438s


In [ ]:
x_cuda = x.cuda()
start = time()
out = knn(x_cuda, x_cuda, k, batch_x=None, batch_y=None, num_workers=1)
print(f'Neighbor search: {time() - start:0.3f}s')
del x_cuda

### GriSPy
```
pip install grispy
```

In [14]:
import grispy as gsp

start = time()
grid = gsp.GriSPy(x.numpy())
dist, ind = grid.nearest_neighbors(x.numpy(), n=k)
print(f'Neighbor search: {time() - start:0.3f}s')

KeyboardInterrupt: 

### FRNN = Final
https://github.com/lxxue/FRNN

In [6]:
from superpoint_transformer.partition.FRNN import frnn
import torch

radius = 1
k_min = 5
k_feat = 50
k_adjacency = 10

torch.cuda.synchronize()
start = time()

# KNN on GPU. frnn_grid_points(x_QUERY, x_TARGET, ...) syntax
distances, neighbors, _, _ = frnn.frnn_grid_points(data.pos.view(1, -1, 3).cuda(), data.pos.view(1, -1, 3).cuda(), None, None, k_feat + 1, radius, grid=None, return_nn=False, return_sorted=True)

# Remove each point from its own neighborhood
neighbors = neighbors[0][:, 1:].cpu()
distances = distances[0][:, 1:].cpu()

# Get the number of found neighbors for each point
n_found_nn = (neighbors != -1).sum(dim=1)

# Identify points which have less than k_min neighbors within R. Those
# will be given 0-features
idx_isolated = torch.where(n_found_nn < k_min)[0]

# Identify points which have more than k_min and less than k neighbors
# within R. For those, we oversample the neighbors to reach k_feat
idx_partial = torch.where((k_min <= n_found_nn) & (n_found_nn < k_feat))[0]
neighbors_partial = neighbors[idx_partial]
distances_partial = distances[idx_partial]

# Since the neighbors are sorted by increasing distance, the missing 
# neighbors will always be the last ones. This helps finding their 
# number and position, for oversampling.

# For each missing neighbor, compute the size of the discrete set to 
# oversample from.
n_valid = torch.repeat_interleave(n_found_nn[idx_partial], k_feat - n_found_nn[idx_partial])

# Compute the oversampling row indices.
idx_x_sampling = torch.repeat_interleave(
    torch.arange(neighbors_partial.shape[0]), k_feat - n_found_nn[idx_partial])

# Compute the oversampling column indices. The 0..9999 factor is a 
# security to handle the case where torch.rand is to close to 1.0, which
# would yield incorrect sampling coordinates that would in result in 
# sampling '-1' indices (ie all we are trying to avoid here)
idx_y_sampling = (n_valid * torch.rand(n_valid.shape[0]) * 0.9999).floor().long()

# Apply the oversampling
idx_missing = torch.where(neighbors_partial == -1)
neighbors_partial[idx_missing] = neighbors_partial[idx_x_sampling, idx_y_sampling]
distances_partial[idx_missing] = distances_partial[idx_x_sampling, idx_y_sampling]

# Restore the oversampled neighborhods with the rest
neighbors[idx_partial] = neighbors_partial
distances[idx_partial] = distances_partial

# Store the neighbors and distances as a Data object attribute
data.neighbors = neighbors
data.distances = distances

torch.cuda.synchronize()
print(f'Neighbor search: {time() - start:0.3f}s')

# IMPORTANT !!!
#   - points with no neighbors within radius -> set to 0-feature !

Neighbor search: 0.346s


In [76]:
idx.min(), (idx == -1).sum() / idx.numel() * 100, (idx.min(dim=1).values == -1).sum() / idx.shape[0] * 100

(tensor(-1, device='cuda:0'),
 tensor(0.0010, device='cuda:0'),
 tensor(0.0025, device='cuda:0'))

In [ ]:
def arange_interleave(sizes):
    """
    Vectorized equivalent of:
        ```torch.cat([torch.arange(x) for x in sizes])```
    """
    assert sizes.dim() == 1, 'Only supports 1D tensors'
    assert isinstance(sizes, torch.LongTensor), 'Only supports LongTensors'
    assert sizes.ge(0).all(), 'Only supports positive integers'
    
    a = torch.cat((torch.LongTensor([0]), sizes[:-1]))
    b = torch.cumsum(a, 0).long()
    return torch.arange(sizes.sum()) - torch.repeat_interleave(b, sizes)

### Pytorch3D
```
pip install -U fvcore
pip install -U iopath
pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu113_pyt1110/download.html
```

In [25]:
import pytorch3d

start = time()
pytorch3d.ops.knn_points(x.view(1, -1, 3), x.view(1, -1, 3), K=k)
print(f'Neighbor search: {time() - start:0.3f}s')

ModuleNotFoundError: No module named 'pytorch3d'

So it seems FRNN on GPU is the clear winner here !

# Geometric features computation

In [7]:
xyz = data.pos
k_geof = 5

### SPG C implem

In [ ]:
import superpoint_transformer.partition.utils.libpoint_utils as point_utils

start = time()
geof = point_utils.compute_geof(xyz.numpy(), neighbors.flatten().numpy().astype('uint32'), k_geof).astype('float32')
print(f'Geometric features: {time() - start:0.3f}s')

Out of memory on my local machine ? Considered doing a chunked version to manage memory but it seems torch's CPU version can handle all once and quite fast

### TP3D-based

In [8]:
import torch
from torch_geometric.data import Data

def batch_pca(xyz):
    """
    Compute the PCA of a batch of point clouds of size (*, N, M).
    """
    assert 2 <= xyz.dim() <= 3
    xyz = xyz.unsqueeze(0) if xyz.dim() == 2 else xyz

    pos_centered = xyz - xyz.mean(dim=1).unsqueeze(1)
    cov_matrix = pos_centered.transpose(1, 2).bmm(pos_centered) / pos_centered.shape[1]
    eigenval, eigenvect = torch.linalg.eigh(cov_matrix)

    # If Nan values are computed, return equal eigenvalues and
    # Identity eigenvectors
    idx_nan = torch.where(torch.logical_and(
        eigenval.isnan().any(1), eigenvect.flatten(1).isnan().any(1)))
    eigenval[idx_nan] = torch.ones(3, dtype=eigenval.dtype, device=xyz.device)
    eigenvect[idx_nan] = torch.eye(3, dtype=eigenvect.dtype, device=xyz.device)

    # Precision errors may cause close-to-zero eigenvalues to be
    # negative. Hard-code these to zero
    eigenval[torch.where(eigenval < 0)] = 0

    return eigenval, eigenvect


class PCAComputePointwise(object):
    """
    Compute PCA for the local neighborhood of each point in the cloud.

    Input data is expected to be stored in DENSE format.

    Results are saved in `eigenvalues` and `eigenvectors` attributes.
    `data.eigenvalues` is a tensor
    :math:`(\lambda_1, \lambda_2, \lambda_3)` such that
    :math:`\lambda_1 \leq \lambda_2 \leq \lambda_3`.
    `data.eigenvectors` is 1x9 tensor containing the eigenvectors
    associated with `data.eigenvalues`, concatenated in the same order.

    Parameters
    ----------
    num_neighbors: int, optional
        Controls the maximum number of neighbors on which to compute
        PCA. If `r=None`, `num_neighbors` will be used as K for
        K-nearest neighbor search. Otherwise, `num_neighbors` will be
        the maximum number of neighbors used in radial neighbor search.
    r: float, optional
        If not `None`, neighborhoods will be computed with a
        radius-neighbor approach. If `None`, K-nearest neighbors will
        be used.
    use_full_pos: bool, optional
        If True, the neighborhood search will be carried on the point
        positions found in the `data.full_pos`. An error will be raised
        if data carries no such attribute. See `GridSampling3D` for
        producing `data.full_pos`.
        If False, the neighbor search will be computed on `data.pos`.
    use_cuda: bool, optional
        If True, the computation will be carried on CUDA.
    workers: int, optional
        If not `None`, the features computation will be distributed
        across the provided number of workers.
    """

    def __init__(
            self, num_neighbors=40, r=None, use_full_pos=False, use_cuda=False,
            use_faiss=True, ncells=None, nprobes=10, chunk_size=1000000):
        self.num_neighbors = num_neighbors
        self.r = r
        self.use_full_pos = use_full_pos
        self.use_cuda = use_cuda and torch.cuda.is_available()
        self.use_faiss = use_faiss and torch.cuda.is_available()
        self.ncells = ncells
        self.nprobes = nprobes
        self.chunk_size = chunk_size

    def _process(self, data: Data):
        assert getattr(data, 'pos', None) is not None, \
            "Data must contain a 'pos' attribute."
        assert not self.use_full_pos \
               or getattr(data, 'full_pos', None) is not None, \
            "Data must contain a 'full_pos' attribute."

        # Recover the query and search clouds
        xyz_query = data.pos
        xyz_search = data.full_pos if self.use_full_pos else data.pos

        # Move computation to CUDA if required
        input_device = xyz_query.device
        if self.use_cuda and not xyz_query.is_cuda and not self.use_faiss:
            xyz_query = xyz_query.cuda()
            xyz_search = xyz_search.cuda()

        # Compute the neighborhoods
        if self.r is not None:
            # Radius-NN search with torch_points_kernel
            sampler = RadiusNeighbourFinder(
                self.r, self.num_neighbors, conv_type='DENSE')
            neighbors = sampler.find_neighbours(
                xyz_search.unsqueeze(0), xyz_query.unsqueeze(0))[0]
        elif self.use_faiss:
            # K-NN search with FAISS
            nn_finder = FAISSGPUKNNNeighbourFinder(
                self.num_neighbors, ncells=self.ncells, nprobes=self.nprobes)
            neighbors = nn_finder(xyz_search, xyz_query, None, None)
        else:
            # K-NN search with KeOps. If the number of points is greater
            # than 16 millions, KeOps requires double precision.
            xyz_query = xyz_query.contiguous()
            xyz_search = xyz_search.contiguous()
            if xyz_search.shape[0] > 1.6e7:
                xyz_query_keops = LazyTensor(xyz_query[:, None, :].double())
                xyz_search_keops = LazyTensor(xyz_search[None, :, :].double())
            else:
                xyz_query_keops = LazyTensor(xyz_query[:, None, :])
                xyz_search_keops = LazyTensor(xyz_search[None, :, :])
            d_keops = ((xyz_query_keops - xyz_search_keops) ** 2).sum(dim=2)
            neighbors = d_keops.argKmin(self.num_neighbors, dim=1)
            # raise NotImplementedError(
            #     "Fast K-NN search has not been implemented yet. Please "
            #     "consider using radius search instead.")

        # Compute PCA for each neighborhood
        # Note: this is surprisingly slow on GPU, so better run on CPU
        eigenvalues = []
        eigenvectors = []
        n_chunks = math.ceil(neighbors.shape[0] / self.chunk_size)
        for i in range(n_chunks):
            xyz_neigh_batch = xyz_search[
                neighbors[i * self.chunk_size: (i + 1) * self.chunk_size]]
            eval, evec = batch_pca(xyz_neigh_batch.cpu())
            evec = evec.transpose(2, 1).flatten(1)
            eigenvalues.append(eval)
            eigenvectors.append(evec)
        eigenvalues = torch.cat(eigenvalues, dim=0)
        eigenvectors = torch.cat(eigenvectors, dim=0)

        # Save eigendecomposition results in data attributes
        data.eigenvalues = eigenvalues.to(input_device)
        data.eigenvectors = eigenvectors.to(input_device)

        return data

    def __call__(self, data):
        if isinstance(data, list):
            data = [self._process(d) for d in tq(data)]
        else:
            data = self._process(data)
        return data

    def __repr__(self):
        attr_repr = ', '.join([f'{k}={v}' for k, v in self.__dict__.items()])
        return f'{self.__class__.__name__}({attr_repr})'


class EigenFeatures(object):
    """
    Compute local geometric features based on local eigenvalues and
    eigenvectors.

    The following local geometric features are computed and saved in
    dedicated data attributes: `norm`, `scattering`, `linearity` and
    `planarity`. The formulation of those is inspired from
    "Hierarchical extraction of urban objects from mobile laser
    scanning data" [Yang et al. 2015]

    Data is expected to carry `eigenvectors` and `eigenvectors`
    attributes:
    `data.eigenvalues` is a tensor
    :math:`(\lambda_1, \lambda_2, \lambda_3)` such that
    :math:`\lambda_1 \leq \lambda_2 \leq \lambda_3`.
    `data.eigenvectors` is 1x9 tensor containing the eigenvectors
    associated with `data.eigenvalues`, concatenated in the same order.
    See `PCAComputePointwise` for generating those.

    Parameters
    ----------
    norm: bool, optional
        If True, the normal to the local surface will be computed.
    linearity: bool, optional
        If True, the local linearity will be computed.
    planarity: bool, optional
        If True, the local planarity will be computed.
    scattering: bool, optional
        If True, the local scattering will be computed.
    temperature: float, optional
        If set to a float value, the returned features will be run
        through a scaled softmax with temperature being the scale. Set
        to None by default.
    """

    def __init__(self, norm=True, linearity=True, planarity=True,
                 scattering=True, verticality=True, temperature=None):
        self.norm = norm
        self.linearity = linearity
        self.planarity = planarity
        self.scattering = scattering
        self.verticality = verticality
        self.temperature = temperature

    def _process(self, data: Data):
        assert getattr(data, 'eigenvalues', None) is not None, \
            "Data must contain an 'eigenvalues' attribute."
        assert getattr(data, 'eigenvectors', None) is not None, \
            "Data must contain an 'eigenvectors' attribute."

        if self.norm:
            # The normal is the eigenvector carried by the smallest
            # eigenvalue
            data.norm = data.eigenvectors[:, :3]

        # Eigenvalues: 0 <= l0 <= l1 <= l2
        # Following, [Yang et al. 2015] we use the sqrt of eigenvalues
        v0 = data.eigenvalues[:, 0].sqrt().squeeze()
        v1 = data.eigenvalues[:, 1].sqrt().squeeze()
        v2 = data.eigenvalues[:, 2].sqrt().squeeze() + 1e-6
        
        e0 = eigenvectors[:, :, 0].abs() * eigenvalues[:, [0]]
        e1 = eigenvectors[:, :, 1].abs() * eigenvalues[:, [1]]
        e2 = eigenvectors[:, :, 2].abs() * eigenvalues[:, [2]]
        u = e0 + e1 + e2

        # Compute the eigen features
        linearity = (v2 - v1) / v2
        planarity = (v1 - v0) / v2
        scattering = v0 / v2
        verticality = u[:, 2] / torch.linalg.norm(u, dim=1)

        # Compute the softmax version of the features, for more
        # opinionated geometric information. As a heuristic, set
        # temperature=5 for clouds of 30 points or more.
        if self.temperature:
            values = (self.temperature * torch.cat([
                linearity.view(-1, 1),
                planarity.view(-1, 1),
                scattering.view(-1, 1)], dim=1)).exp()
            values = values / values.sum(dim=1).view(-1, 1)
            linearity = values[:, 0]
            planarity = values[:, 1]
            scattering = values[:, 2]

        if self.linearity:
            data.linearity = linearity

        if self.planarity:
            data.planarity = planarity

        if self.scattering:
            data.scattering = scattering
        
        if self.verticality:
            data.verticality = verticality

        return data

    def __call__(self, data):
        if isinstance(data, list):
            data = [self._process(d) for d in data]
        else:
            data = self._process(data)
        return data

    def __repr__(self):
        attr_repr = ', '.join([f'{k}={v}' for k, v in self.__dict__.items()])
        return f'{self.__class__.__name__}({attr_repr})'


In [ ]:
# On GPU
xyz = torch.rand(10**6, 50, 3).cuda()

torch.cuda.synchronize()
start = time()
batch_pca(xyz)
torch.cuda.synchronize()
print(f'PCA: {time() - start:0.3f}s')

PCA: 54.819s


In [ ]:
# On CPU
xyz = torch.rand(10**6, 50, 3)

start = time()
batch_pca(xyz)
print(f'PCA: {time() - start:0.3f}s')

PCA: 1.981s


In [156]:
from torch_geometric.data import Data

# On CPU
xyz = torch.rand(10**6, 50, 3)

start = time()
eigenvalues, eigenvectors = batch_pca(xyz)
print(f'PCA CPU: {time() - start:0.3f}s')

# On CPU
start = time()
d = Data(x=xyz, eigenvalues=eigenvalues, eigenvectors=eigenvectors)
d = EigenFeatures(norm=True, linearity=True, planarity=True, scattering=True, verticality=True, temperature=None)(d)
print(f'Geometric Features: {time() - start:0.3f}s')

PCA CPU: 2.021s
Geometric Features: 0.012s


In [143]:
from torch_geometric.data import Data

# On CPU
xyz = torch.rand(10**6, 50, 3)

start = time()
eigenvalues, eigenvectors = batch_pca(xyz)
print(f'PCA CPU: {time() - start:0.3f}s')

# On GPU
torch.cuda.synchronize()
start = time()
d = Data(x=xyz.cuda(), eigenvalues=eigenvalues.cuda(), eigenvectors=eigenvectors.cuda())
d = EigenFeatures(norm=True, linearity=True, planarity=True, scattering=True, verticality=True, temperature=None)(d)
torch.cuda.synchronize()
print(f'Geometric Features: {time() - start:0.3f}s')

PCA CPU: 2.037s
Geometric Features: 0.137s


Surprisingly, torch's CPU implementation is faster both for computing PCA and geometric features is faster on CPU overall !

### Final

In [9]:
from torch_geometric.data import Data

# On CPU
start = time()
eigenvalues, eigenvectors = batch_pca(data.pos[neighbors])
data.eigenvalues = eigenvalues
data.eigenvectors = eigenvectors
print(f'PCA CPU: {time() - start:0.3f}s')

# On CPU
start = time()
data = EigenFeatures(norm=True, linearity=True, planarity=True, scattering=True, verticality=True, temperature=None)(data)
print(f'Geometric Features: {time() - start:0.3f}s')

PCA CPU: 0.408s
Geometric Features: 0.012s


# Point adjacency graph computation
This graph is based on the nearest neighbor graph computed for geometric features. However, although features may require 30-50 neighbors to produce good partition, the adjacency graph benefits from using fewer neighbors (eg 10 in the paper).

In [10]:
from superpoint_transformer.partition.FRNN import frnn
import torch

radius = 1
k_min = 5
k_feat = 50
k_adjacency = 10
lambda_edge_weight = 1

In [11]:
start = time()

# Create the adjacency graph edges. NB: this graph is directed wrt 
# Pytorch Geometric, but cut-pursuit is happy with it
source = torch.arange(data.num_nodes).repeat_interleave(k_adjacency)
target = data.neighbors[:, :k_adjacency].flatten()
data.edge_index = torch.stack((source, target))

# Create the edge features for the adjacency graph
distances = data.distances[:, :k_adjacency].flatten()
data.edge_attr = 1 / (lambda_edge_weight + distances / distances.mean())

# Gather the pointwise features that will be used for the partition
# NB: we use a heuristic to increase the importance of verticality
data.x = torch.column_stack((data.linearity.view(-1, 1), data.planarity.view(-1, 1), data.scattering.view(-1, 1), data.verticality.view(-1, 1) * 2, data.rgb))

print(f'Geometric Features: {time() - start:0.3f}s')

Geometric Features: 0.010s


# Partition

In [19]:
sys.path.append(os.path.join(file_path, "superpoint_transformer/partition/grid_graph/python/bin"))
sys.path.append(os.path.join(file_path, "superpoint_transformer/partition/parallel_cut_pursuit/python/wrappers"))

from grid_graph import edge_list_to_forward_star
from cp_kmpp_d0_dist import cp_kmpp_d0_dist

def pcp(data, reg_strength, cutoff=1, parallel=True, balance=True):
    """Partition the graph with parallel cut pursuit."""
    # Recover needed tensors from Data object
    x = np.asfortranarray(data.x.numpy().T)
    source = data.edge_index[[0]].numpy()
    target = data.edge_index[[0]].numpy()
    edge_weight = data.edge_attr.numpy()
    
    # Convert to forward-star graph representation
    first_edge, adj_vertices, reindex = edge_list_to_forward_star(
        data.num_nodes, data.edge_index.T.contiguous().numpy())
    first_edge = first_edge.astype('uint32')
    adj_vertices = adj_vertices.astype('uint32')
        
    if parallel:
        max_thread = 0
    else:
        max_thread = 1
    
#     if return_intermediate:
#         Comp, rX, it, Obj, Time, comp_List = cp_kmpp_d0_dist(
#             1, x, first_edge, adj_vertices,
#             edge_weights=reg_strength * edge_weight[reindex], min_comp_weight=cutoff,
#             cp_dif_tol=1e-2, cp_it_max=10, split_damp_ratio=0.7, verbose=False, 
#             max_num_threads=max_thread, compute_Com=True, compute_Obj=True, compute_Time=True,
#             balance_parallel_split=balance)
#         return comp_List, Comp, (Obj, Time, rX)
#     else:
#         Comp, rX, it, comp_List = cp_kmpp_d0_dist(
#             1, x, first_edge, adj_vertices,
#             edge_weights=reg_strength * edge_weight[reindex], min_comp_weight=cutoff,
#             cp_dif_tol=1e-2, cp_it_max=10, split_damp_ratio=0.7, verbose=False,
#             max_num_threads=max_thread, compute_Com=True, balance_parallel_split=balance)
#         return comp_List, Comp, []    
    
    #p2sp, sp_x, sp2p, Time
    out = cp_kmpp_d0_dist(
            1, x, first_edge, adj_vertices,
            edge_weights=reg_strength * edge_weight[reindex], min_comp_weight=cutoff,
            cp_dif_tol=1e-2, cp_it_max=10, split_damp_ratio=0.7, verbose=False, 
            max_num_threads=max_thread, compute_Time=True, compute_List=True,
            balance_parallel_split=balance)
    return out

In [ ]:
pcp(data, 0.5, cutoff=10)

questions
- cutoff 1 ?
- max threads 1 ?


In [ ]:
# PCP computation
components2, in_component2, tracks_single = pcp(features, graph_nn, args.reg_strength, 10)
# en sachant que pour graph_nn tu n'as besoin que de graph_nn["source"], graph_nn["target"] et graph_nn["edge_weight"]

# Superpoint graph computation
In the original SPG implementation, the SP graph would be computed based on the pointwise Delaunay triangulation graph. This is super inefficient. Instead, we will compute the Delaunay triangulation on the superpoint level, which sould be much faster. However, to account for large and long-shaped superpoints, we will not work with the SP centroids only (Delaunay triangulation would not capture all adjacent SPs), but on random/farthest point samplings inside the SPs (as function of SP area/volume/number of points).  